In [1]:
from keras.applications.vgg16 import VGG16
import os
from keras.layers import Flatten, Dense, AveragePooling2D,GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
vgg16_base = VGG16(include_top=False, weights='imagenet',
                    input_tensor=None, input_shape=(150, 150,3))
# Note that the preprocessing of InceptionV3 is:
# (x / 255 - 0.5) x 2

print('Adding Average Pooling Layer and Softmax Output Layer ...')
output = vgg16_base.get_layer(index = -1).output  # Shape: (8, 8, 2048)
#output = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(output)
#output = Flatten(name='flatten')(output)
#output = Dense(8, activation='softmax', name='predictions')(output)
output = GlobalAveragePooling2D()(output)
# let's add a fully-connected layer
output = Dense(1024, activation='relu')(output)
# and a logistic layer -- let's say we have 200 classes
output = Dense(8, activation='softmax')(output)


vgg16_model = Model(vgg16_base.input, output)

Adding Average Pooling Layer and Softmax Output Layer ...


In [3]:
vgg16_model.load_weights("weights.h5")

In [ ]:





test_dir = "test_stg1/"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
            test_dir,
            target_size=(150,150),
            batch_size=10,
            class_mode=None,
            shuffle=False)

preds = vgg16_model.predict_generator(test_generator,1000)



Found 1000 images belonging to 1 classes.


In [5]:
preds = np.clip(preds, 0.12,0.95)

In [ ]:

submission = pd.DataFrame(test_preds, columns= os.listdir(train_dir))
test_files = os.listdir("test_stg1/test_stg1/")
submission.insert(0, 'image', test_files)
submission.head()